In [ ]:
%%cmd
@echo off

REM Name of algo -> ECR
set algorithm_name=docker-cicd

#docker file location change

cd C:\Users\70071865\Documents\mlops\mlops\blog

REM make serve executable change here
icacls C:\Users\70071865\Documents\mlops\mlops\blog\serve /grant Users:F
    
for /f "tokens=1" %a in ('aws sts get-caller-identity --query Account --output text') do set account=%a

REM Region, defaults to ap-south-1
for /f "tokens=1" %a in ('aws configure get region') do set region=%a
if "%region%"=="" set region=ap-south-1

set fullname=%account%.dkr.ecr.%region%.amazonaws.com/%algorithm_name%:v1

REM If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "%algorithm_name%" > nul 2>&1

if NOT %ERRORLEVEL%==0 (
    aws ecr create-repository --repository-name %algorithm_name% --image-scanning-configuration scanOnPush=true > nul
)

REM Get the login command from ECR and execute it directly
aws ecr get-login-password --region %region% | docker login --username AWS --password-stdin %fullname%

REM Build the docker image locally with the image name and then push it to ECR
REM with the full name.
docker build -t %algorithm_name% .
docker tag %algorithm_name% %fullname%
docker push %fullname%

In [ ]:
%%cmd
get-access-key-info
docker push %fullname%